In [1]:
from datasets import FrameImageDataset, FrameVideoDataset

image_dataset = FrameImageDataset()
video_dataset = FrameVideoDataset()

In [5]:
image_dataset.label_to_action

{0: 'BodyWeightSquats',
 1: 'HandstandPushups',
 2: 'HandstandWalking',
 4: 'JumpRope',
 3: 'JumpingJack',
 5: 'Lunges',
 6: 'PullUps',
 7: 'PushUps',
 8: 'TrampolineJumping',
 9: 'WallPushups'}

In [6]:
from datetime import datetime

def get_timestamp() -> str:
    now = datetime.now()
    return now.strftime("%Y%m%d%H%M%S%f")[:-3]  # [:-3] to get milliseconds instead of microseconds

In [7]:
get_timestamp()

'20251002222934094'